### Tag anomaly record using 3 sigma rule

In [1]:
import os
parent_dir = os.path.dirname(os.path.abspath('./'))
filename = f"{parent_dir}/input_data/Electric_Production.csv"
print(f"Filename: {filename}")

Filename: /Users/sristiraj/Downloads/sristiraj/ML/anomaly_detection/timeseries/input_data/Electric_Production.csv


### Read data using duckdb

In [2]:
import duckdb

elec_df = duckdb.read_csv(filename)

In [5]:
query_str = '''
    select * 
    from (
        select *, 
            avg(IPG2211A2N) over(order by period ROWS between 11 PRECEDING AND CURRENT ROW) roll_avg, 
            stddev(IPG2211A2N) over(order by period ROWS between 11 PRECEDING AND CURRENT ROW) roll_std,
            row_number() over (order by period) rn
            from 
                (select 
                    year(date)*100+month(date) period,
                    sum(IPG2211A2N) IPG2211A2N
                    from
                    elec_df
                    group by year(date)*100+month(date)
                ) k
        )
    where rn >= 12
    and abs(IPG2211A2N-roll_avg) >= 3*roll_std -- 3 sigma rule
'''

In [6]:
duckdb.sql(query_str)


┌────────┬────────────┬────────────────────┬────────────────────┬───────┐
│ period │ IPG2211A2N │      roll_avg      │      roll_std      │  rn   │
│ int64  │   double   │       double       │       double       │ int64 │
├────────┼────────────┼────────────────────┼────────────────────┼───────┤
│ 201801 │   129.4048 │ 1131.9812833333333 │ 316.13076686143324 │    34 │
└────────┴────────────┴────────────────────┴────────────────────┴───────┘